# **FINE TUNE deepset/roberta-base-squad2**

In [3]:
pip install huggingface_hub

In [53]:
!pip install tqdm

In [4]:
pip install PyMuPDF pandas openpyxl

In [5]:
!pip install datasets

In [6]:
pip install pdfplumber

In [7]:
!pip install transformers

In [1]:
!pip install bitsandbytes

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [59]:
from tqdm import tqdm
from typing import Dict

In [8]:
import fitz
import torch
import json
import os
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments ,pipeline
from datasets import Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
import bitsandbytes as bnb

L fonction hadhi pour n extractiw les text mn les pdf

In [15]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

In [16]:
pdf_folder = "/content/drive/MyDrive/rapport expertises 2023 to 2024"
pdf_texts = {file: extract_text_from_pdf(os.path.join(pdf_folder, file)) for file in os.listdir(pdf_folder) if file.endswith(".pdf")}

l hna bach n definow structure data mta3na w naetiw des exemple 7attachi yefhem kifach y5arej les donnee

In [17]:
train_data={
  "data": [
    {
      "title": "Rapport d'Expertise Radiologique",
      "paragraphs": [
        {
          "context": "Suite à la Demande, datée du 12/12/2022, de la Société EMBALLAGE TUBULAIRE, sise à Bouargoub...",
          "qas": [
            {
              "question": "Quelle est la date de la demande ?",
              "id": "1",
              "answers": [{"text": "12/12/2022", "answer_start": 32}]
            },
            {
              "question": "Quel est le nom de la société concernée ?",
              "id": "2",
              "answers": [{"text": "EMBALLAGE TUBULAIRE", "answer_start": 52}]
            },
            {
              "question": "Où se trouve la société ?",
              "id": "3",
              "answers": [{"text": "Bouargoub", "answer_start": 77}]
            }
          ]
        },
        {
          "context": "Le contrôle radiologique a été effectué le 21/12/2022...",
          "qas": [
            {
              "question": "Quelle est la date du contrat ?",
              "id": "4",
              "answers": [{"text": "21/12/2022", "answer_start": 39}]
            }
          ]
        },
        {
          "context": "Le lot contrôlé pèse 10003Kg...",
          "qas": [
            {
              "question": "Quel est le poids du lot ?",
              "id": "5",
              "answers": [{"text": "10003Kg", "answer_start": 21}]
            }
          ]
        },
        {
          "context": "Le lot a été chargé immédiatement après contrôle dans un conteneur sous référence GSU 180190 22 G1...",
          "qas": [
            {
              "question": "Quel est le numéro de référence du conteneur ?",
              "id": "6",
              "answers": [{"text": "GSU 180190 22 G1", "answer_start": 85}]
            }
          ]
        },
        {
          "context": "Equipement utilisé pour le contrôle : Radiomètre type « FH 40 » avec sonde externe GAMMA et Rx, MCB2, contaminamètre...",
          "qas": [
            {
              "question": "Quels équipements ont été utilisés pour le contrôle ?",
              "id": "7",
              "answers": [{"text": "Radiomètre type « FH 40 », sonde externe GAMMA et Rx, MCB2, contaminamètre", "answer_start": 37}]
            }
          ]
        },
        {
          "context": "Bruit de fond enregistré : Max 0.8 c/s « MCB2 », Max 90 nSv/h « FH 40 »...",
          "qas": [
            {
              "question": "Quel est le bruit de fond enregistré ?",
              "id": "8",
              "answers": [{"text": "Max 0.8 c/s « MCB2 », Max 90 nSv/h « FH 40 »", "answer_start": 27}]
            }
          ]
        },
        {
          "context": "Résultats enregistrés : Entre 0.4 c/s et 1.2 c/s « MCB2 », Entre 33 nSv/h et 160 nSv/h « FH 40 »...",
          "qas": [
            {
              "question": "Quels sont les résultats enregistrés ?",
              "id": "9",
              "answers": [{"text": "Entre 0.4 c/s et 1.2 c/s « MCB2 », Entre 33 nSv/h et 160 nSv/h « FH 40 »", "answer_start": 27}]
            }
          ]
        },
        {
          "context": "Conclusion : Les mesures effectuées ne montrent pas un écart significatif des valeurs par rapport à celles du bruit de fond naturel...",
          "qas": [
            {
              "question": "Quelle est la conclusion de l'expertise ?",
              "id": "10",
              "answers": [{"text": "Les mesures effectuées ne montrent pas un écart significatif des valeurs par rapport à celles du bruit de fond naturel.", "answer_start": 12}]
            }
          ]
        },
        {
          "context": "Directeur Général : Nafaa REGUIGUI...",
          "qas": [
            {
              "question": "Qui est le Directeur Général ?",
              "id": "11",
              "answers": [{"text": "Nafaa REGUIGUI", "answer_start": 21}]
            }
          ]
        },
        {
          "context": "Nom / Prénom du Rédacteur : Wahid Mejri...",
          "qas": [
            {
              "question": "Qui a rédigé le rapport ?",
              "id": "12",
              "answers": [{"text": "Wahid Mejri", "answer_start": 27}]
            }
          ]
        },
        {
          "context": "Nom / Prénom du Vérificateur/Approbateur : Mr. NAFAA REGUIGUI...",
          "qas": [
            {
              "question": "Qui a vérifié et approuvé le rapport ?",
              "id": "13",
              "answers": [{"text": "Mr. NAFAA REGUIGUI", "answer_start": 47}]
            }
          ]
        },
        {
          "context": "Organisme Assisté : Société EMBALLAGE TUBULAIRE...",
          "qas": [
            {
              "question": "Quel est l'organisme assisté ?",
              "id": "14",
              "answers": [{"text": "Société EMBALLAGE TUBULAIRE", "answer_start": 20}]
            }
          ]
        }
      ]
    }
  ]
}


l format hadhiye pour que n definiw data mta3ne avec les structure des data de hugging face 5ater l etape hadhe lazem

In [18]:
dataset = Dataset.from_list(train_data["data"])

lehna 9a3ed n importi f l modele eli bach na3mlo l finetuning

In [19]:
MODEL_NAME = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

l fonction hadhi bach t5alina n conserviw format barka ye5dem 3liha f les pdf lkol

In [27]:
def preprocess_data(example):
    processed_data = {'input_ids': [], 'attention_mask': []}
    for paragraph in example['paragraphs']:
        for qa in paragraph['qas']:
            tokenized_data = tokenizer(
                qa["question"],
                paragraph["context"],
                truncation=True,
                padding="max_length",
                max_length=512
            )
            processed_data['input_ids'].append(tokenized_data['input_ids'])
            processed_data['attention_mask'].append(tokenized_data['attention_mask'])
    return processed_data

lena appliquna la fonction 3a data mta3na

In [ ]:
dataset = dataset.map(preprocess_data)

lena just definition ll model

In [29]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

hadhoma des parametre de train

In [30]:
training_args = TrainingArguments(
    output_dir="./qa_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


lena bach n defiiw un variable mn la classe train n7o6o f westo esm l model w dataset w l parametre de l'entrainement

In [37]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

In [35]:
!pip install bitsandbytes --upgrade

lena lancit le train wela tnejem t9ol le finetune

In [37]:
trainer.train()

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

lena aemlt save llmodel

In [ ]:
trainer.save_model("./qa_model_trained")

lehna aemlt save ll parametre et les fichier du model

In [40]:
tokenizer.save_pretrained("./qa_model_trained")

('./qa_model_trained/tokenizer_config.json',
 './qa_model_trained/special_tokens_map.json',
 './qa_model_trained/vocab.json',
 './qa_model_trained/merges.txt',
 './qa_model_trained/added_tokens.json',
 './qa_model_trained/tokenizer.json')

w lena bach nebda n appliquih 3al les PDF

# **GENERATE THE MODEL TO ALL OF THE PDFs**

In [56]:
PDF_FOLDER = "/content/drive/MyDrive/rapport expertises 2023 to 2024"
OUTPUT_CSV = "extracted_answers.csv"

lena initialisit l modele eli fintunee

In [47]:
qa_pipeline = pipeline("question-answering", model="./qa_model_trained", tokenizer="./qa_model_trained")

Device set to use cpu


hadhoma ales colone de notre data

In [48]:
columns = [
    'Date de la demande', 'Nom de la societe', 'Localisation des ateliers',
    'Poids du lot', 'Date du contrat', 'N580582 ',
    'equipement utiliser  pour le contrat', 'Bruit de fond enregistre',
    'Resultats enregistr', 'Conclusion', 'equiuipe technique',
    'Directeur generale', 'Realise par', 'Nom /  prenom du predacteur',
    'Technicien en Chef ', 'Nom  / prenom du verficateur/approbateur',
    'Fonction/grade du verificateur/approbateur', 'Objectif',
    'Organisme Assisité'
]

fonction hadhi takra les donnes eli f les pdfs

In [49]:
def lire(pdf_path: str) -> str:
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text() + "\n"
    return text.strip()

hadhiya bach t extracti les  diferents chmps eli f west l pdf 5ater fema des tableau des images w des espaces far9in

In [60]:
def extract_fields_from_pdf(text: str) -> Dict[str, str]:
    extracted = {}
    for col in columns:
        question = f"Quel est le champ suivant : {col} ?"
        answer = extract_field(question, text)
        extracted[col] = answer if answer else "Non trouvé"
    return extracted

hadhiya bach t separi les champs w taetihm ll model

In [61]:
def separer(question: str, context: str) -> str:
    result = qa_pipeline(question=question, context=context)
    return result.get('answer', '').strip()

whadiya t 7ot les donnes f west l fichier

In [44]:
def traiter(pdf_folder: str, output_csv: str):
    """Traiter tous les PDF d'un dossier"""
    data_rows = []

    pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

    for pdf_file in pdf_files:
        print(f"🔄 Traitement de : {pdf_file}")
        pdf_path = os.path.join(pdf_folder, pdf_file)
        text = read_pdf_text(pdf_path)
        extracted_data = extract_fields_from_pdf(text)
        data_rows.append(extracted_data)

    df = pd.DataFrame(data_rows, columns=columns)
    df.to_csv(output_csv, index=False)
    print(f"✅ CSV généré : {output_csv}")

# **END**